# GRASS Tool with Parameters

## Interface Definition for Scripts

Since GRASS tools are executable scripts (or generally programs), the interface of a GRASS tool is the command line interface of a Python script. A dedicated function _grass.script.parser_ takes care of processing the command line arguments based on the interface description specified in a Python comment with a key-value syntax defined by GRASS GIS.

The following is an example of a script which takes two parameters: name of a vector map and name of a raster map:

In [62]:
%%writefile viewshed.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: viewshed
# % keyword: geometry
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# %end
# %option G_OPT_M_COORDS
# %end
# %option G_OPT_R_INPUTS
# %end
# %option G_OPT_F_OUTPUT
# %end

import subprocess
import sys
import csv
import io

import grass.script as gs


def viewshed(
    elevation,
    coords,
    output,
    sample_continuous,
):
    name = "viewshed"
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=name,
        coordinates=coords,
        flags="cb",
    )
    gs.run_command("r.null", map=name, setnull=0)
    results = {}
    results["x"] = coords[0]
    results["y"] = coords[1]
    for each in sample_continuous:
        table_data = gs.read_command(
            "r.univar",
            map=each,
            zones=name,
            quiet=True,
            flags="t",
            separator="comma",
        )
        # While we could use .strip().splitlines()[-1].split(",") here,
        # using a proper CSV reader is more robust.
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            for key, value in row.items():
                new_key = f"{each}_{key}"
                results[new_key] = value
    print(results)


def main():
    options, flags = gs.parser()
    coords = options["coordinates"].split(",")
    sample_continuous = options["input"].split(",")
    viewshed(
        elevation=options["input"],
        coords=(float(coords[0]), float(coords[1])),
        output=options["output"],
        sample_continuous=sample_continuous,
    )


if __name__ == "__main__":
    main()

Overwriting vector_to_raster.py


As before, we will make the script executable:

In [16]:
!chmod u+x vector_to_raster.py

Running the script with `--help` gives its interface described for command line use:

In [63]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./vector_to_raster.py elevation=elevation coordinates=637_657,222_899 output="data.txt" input=elevation --o

Starting GRASS GIS...
Cleaning up temporary files...
Executing <./vector_to_raster.py elevation=elevation coordinates=637_657,222_899 output=data.txt input=elevation --o> ...
Computing events...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Computing visibility...
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
Writing output raster map...
   0   6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 100
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
{'elevation_zone': '1', 'elevation_label': '', 'elevation_non_null_cells': '121', 'elevation_null_cells': '0', 'elevation_min': 

Running the script with `--html-description` gives the command line interface described in HTML which later becomes a part of the tool's HTML documentation:

In [51]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec r.univar map=elevation zones=viewshed -t separator=comma

Starting GRASS GIS...
Cleaning up temporary files...
Executing <r.univar map=elevation zones=viewshed -t separator=comma> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
zone,label,non_null_cells,null_cells,min,max,range,mean,mean_of_abs,stddev,variance,coeff_var,sum,sum_abs
1,,121,0,65.0054626464844,152.262680053711,87.2572174072266,115.819097786895,115.819097786895,20.486681456427,419.704117097109,17.6885175656626,14014.1108322144,14014.1108322144
Execution of <r.univar map=elevation zones=viewshed -t separator=comma> finished.
Cleaning up temporary files...


## Task: Add JSON or CSV Output

### JSON

```
json
```

### CSV

```
csv.DictWriter()
```

In [ ]:
grass --tmp-location XY --exec v.db.select --script

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec r.univar map=elevation zones=viewshed -t separator=comma format=json